In [1]:
import os
import re
import json
import time
import folium
import random
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup
from selenium import webdriver
import matplotlib.pyplot as plt
from openpyxl.workbook import Workbook
from matplotlib import font_manager, rc
plt.rc('font', family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False
from selenium.webdriver.common.by import By
import platform
if platform.system()  == "Windows" :
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname = path).get_name()
    rc("font", family = font_name)

In [2]:
# 원자료 불러오기
df17 = pd.read_csv('./data_org/2017년_원자료.csv', encoding = 'euc-kr')
df18 = pd.read_csv('./data_org/2018년_원자료.csv', encoding = 'euc-kr')
df19 = pd.read_csv('./data_org/2019년_원자료.csv')

# 변수정리 엑셀 불러오기
df_col = pd.read_excel('./data_org/데이터_변수정리_찐최종본.xlsx',
                       header =1, usecols=[1,6,14], names = ['val1', 'val2', 'val3'])

df17_col = pd.read_excel('./data_org/데이터_변수정리_찐최종본.xlsx',
                         usecols = 'A : B', sheet_name = '변수명매칭')
df18_col = pd.read_excel('./data_org/데이터_변수정리_찐최종본.xlsx',
                         usecols = 'D : E', sheet_name = '변수명매칭')

df17_val = pd.read_excel('./data_org/데이터_변수정리_찐최종본.xlsx',
                         usecols = 'A : B', sheet_name = '17년여행지변수명')
df18_val = pd.read_excel('./data_org/데이터_변수정리_찐최종본.xlsx',
                         usecols = 'A : B', sheet_name = '18년여행지변수명')
df19_val = pd.read_excel('./data_org/데이터_변수정리_찐최종본.xlsx',
                         usecols = 'A : B', sheet_name = '19년여행지변수명')

# 17년 전처리

In [3]:
# 17년 데이터 컬럼 가공
val_lis = list(df_col['val2'].dropna(axis = 0))
val_lis2 = []

for val in val_lis:
    if 'yq9a' in val:
        temp = ['yq9a{}'.format(str(i).zfill(2)) for i in range(1, 18)]
        temp = temp+['yq9a97', 'yq9a99']
        
    elif '~' in val:
        text = re.compile('[0-9]~').search(val).start()
        i_start = re.compile('[0-9]{0,3}$').search(val).start()
        i_end = re.compile('[0-9]{0,3}$').search(val).end()
        ran = val[int(i_start):int(i_end)]
        for i in range(1,int(ran)+1):
            val_lis2.append('{}{}'.format(val[:text],i))
            
    else:
        val_lis2.append(val)

val_lis2 = val_lis2+temp

df17 = df17.loc[:,val_lis2]

# 최초방문인 관광객들 1회방문 넣어주기
df17["wq1a"].fillna(1,inplace=True)

# 방문목적을 19년 기준에 맞게 변경
df17[["q5"]] = df17[["q5"]].replace({1:1,2:6,3:5,4:1,5:2,6:3,7:4,8:6})

# 만족활동 변수명을 기준에 맞게 바꿔주기
df17[["q5a1","q5a2","q5a3"]] = df17[["q5a1","q5a2","q5a3"]].replace({1:3,2:2,3:4,4:1,5:7,6:7,7:5,8:3,9:12,10:11,
                                                                     11:13,12:13,13:6,14:8,15:8,16:10,17:13})

# 17년도 고려요인 무응답을 결측값으로 변경
for i in range(13841) :
    if df17.loc[i, 'q5a1'] == 99 :
        df17.loc[i, 'q5a1'] = np.nan

# 1~10까지 비율에 맞춰 할당할 4번(휴양, 휴식)을 빼고 기준에 맞게 바꿔주기
df17[["q8a1","q8a2","q8a3"]] = df17[["q8a1","q8a2","q8a3"]].replace(4,"n")
df17[["q8a1","q8a2","q8a3"]] = df17[["q8a1","q8a2","q8a3"]].replace({1:2,2:1,3:9,5:9,6:9,7:8,8:8,9:8,10:10,11:10,12:11,13:13,
                                                                     14:12,15:11,16:11,17:4,18:3,19:5,20:6,21:14,22:5,23:14})

# 랜덤할당 함수
def ratelist(row_name, val_rmd):
    li = []
    
    if row_name == 'q8a1':
        dic = {1.0:0.318198, 2.0:0.214775, 3.0:0.134685, 10.0:0.101622,
                4.0:0.092523, 6.0:0.064324, 5.0:0.034054,7.0:0.017477,
                8.0:0.013423, 9.0:0.008919}
    elif row_name == 'q8a2':
        dic = {2.0:0.328732,1.0:0.192887,3.0:0.145144,4.0:0.110057,
                10.0:0.072189,5.0:0.066053,7.0:0.031636,6.0:0.030678,
                8.0:0.011888,9.0:0.010737}
    else:
        dic = {3.0:0.176084,2.0:0.157057,1.0:0.156711,4.0:0.129958,
                10.0:0.127076,5.0:0.111624,6.0:0.054659,7.0:0.047970,
                8.0:0.024331,9.0:0.014530}
    
    rate = list(dic.values())
    total = len(df17[df17[row_name]=="n"])
    count=0

    for i in range(1,11):
        num = int(round(dic[i],len(str(total)))*total)
        count+=num
        for j in range(0, num):
            li.append(i)
            
    if count != total:
        for k in range(0,total-count):
            li.append(int(list(dic.keys())[0]))
            
    random.shuffle(li)
    
    for l, m in zip(range(0,len(li)), df17[df17[row_name]==val_rmd].index):
        df17.loc[m,row_name] = li[l]
        
# 4번("n") 비율별로 나눠주기
ratelist("q8a1","n")
ratelist("q8a2","n")
ratelist("q8a3","n")

# 방문지역(17개시도) 무응답 및 기타 삭제(사유: 없거나 미미한 수치)
del df17["yq9a97"]
del df17["yq9a99"]

# 만족도의 9번(해당 없음)을 19년도처럼 0번으로 교체
for i in range(10) :
    df17 = df17.replace({f'q13a{str(i + 1).zfill(2)}' : 9.0}, 0)

# 결측치가 있는 항목(숙박비,거주국 지불경비) 0으로 채움
df17[["mtzq19a1","mtzq19a2"]] = df17[["mtzq19a1","mtzq19a2"]].fillna(0)

# 19년도처럼 문화+오락 비용 합치기
df17["mtzq19a6+8"] = df17["mtzq19a6"] + df17["mtzq19a8"]

# 문화, 오락 비용 개별 컬럼 제거, 문화+오락 컬럼 사용
del df17['mtzq19a6']
del df17['mtzq19a8']

df17.rename(columns={'mtzq19a6+8':'mtzq19a6'},inplace=True)

# 금액 수치를 19년도의 범주에 맞춰 변경
for c in ["mtzq19a1","mtzq19a2"] :
    for i in range(len(df17)) :
        if df17.loc[i,c] <= 100 :
            df17.loc[i,c] = 1
        elif df17.loc[i,c] > 100 and df17.loc[i,c] <= 200 :
            df17.loc[i,c] = 2
        elif df17.loc[i,c] > 200 and df17.loc[i,c] <= 500 :
            df17.loc[i,c] = 3
        elif df17.loc[i,c] > 500 and df17.loc[i,c] <= 1000 :
            df17.loc[i,c] = 4
        elif df17.loc[i,c] > 1000 and df17.loc[i,c] <= 10000 :
            df17.loc[i,c] = 5
        elif df17.loc[i,c] > 10000 :
            df17.loc[i,c] = 98
            
for c in ["mtzq19a3","mtzq19a5"] :
    for i in range(len(df17)) :
        if df17.loc[i,c] <= 50 :
            df17.loc[i,c] = 1
        elif df17.loc[i,c] > 50 and df17.loc[i,c] <= 100 :
            df17.loc[i,c] = 2
        elif df17.loc[i,c] > 100 and df17.loc[i,c] <= 200 :
            df17.loc[i,c] = 3
        elif df17.loc[i,c] > 200 and df17.loc[i,c] <= 500 :
            df17.loc[i,c] = 4
        elif df17.loc[i,c] > 500 and df17.loc[i,c] <= 10000 :
            df17.loc[i,c] = 5
        elif df17.loc[i,c] > 10000 :
            df17.loc[i,c] = 98
            
for c in ["mtzq19a4","mtzq19a7","mtzq19a10","mtzq19a6"] :
    for i in range(len(df17)) :
        if df17.loc[i,c] == 0 :
            df17.loc[i,c] = 1
        elif df17.loc[i,c] > 0 and df17.loc[i,c] <= 30 :
            df17.loc[i,c] = 2
        elif df17.loc[i,c] > 30 and df17.loc[i,c] <= 100 :
            df17.loc[i,c] = 3
        elif df17.loc[i,c] > 100 and df17.loc[i,c] <= 10000 :
            df17.loc[i,c] = 4
        elif df17.loc[i,c] > 10000 :
            df17.loc[i,c] = 5
            
# 연령대가 9번(기타)인 값들 제거
df17 = df17[df17["age"] != 9].reset_index(drop = True)

# 나이의 범주(0~5)를 19년도처럼 변경(1~6)
df17["age"] = df17["age"] + 1

# 17년 여행지 변수명 한글로 변환
for i in range(813) : # 17년 여행지 변수명의 row 갯수가 813개
    df17 = df17.replace({'q11a1' : df17_val.loc[i, '변수명']}, df17_val.loc[i, '변수내용'])
    df17 = df17.replace({'q11a2' : df17_val.loc[i, '변수명']}, df17_val.loc[i, '변수내용'])
    df17 = df17.replace({'q11a3' : df17_val.loc[i, '변수명']}, df17_val.loc[i, '변수내용'])

# 19년도와 컬럼명 동기화
for i in range(58) :
    df17 = df17.rename(columns = {df17_col.loc[i, '17변수명']: df17_col.loc[i, '19변수명']})

# 방문 지역부분 변수명을 19년도에 맞춰 변경
dic1={}
w_list = [1, 4, 8, 9, 6, 10, 11, 17, 14, 15, 3, 7, 2, 5, 12, 13, 16]

for i in range(1,18) :
    dic1["Q10_1B{}".format(i)] = "Q10_1B{}".format(w_list[i - 1])
    
df17.rename(columns = dic1, inplace = True)

# 월이 들어간 컬럼을 사용해 년-월 컬럼 생성
for i in range(len(df17)) :
    df17.loc[i, "D_YEAR"] = "2017-{}".format(str(int(df17["D_MON"][i])).zfill(2))

# 외래방문객의 방문횟수 이상치 제거
df17 = df17.drop(df17[df17['Q1'] > 300].index).reset_index(drop = True)

# 18년 전처리

In [4]:
# 18년 데이터 컬럼 가공
val_lis = list(df_col['val3'].dropna(axis = 0))
val_lis2 = []

for val in val_lis:
    if '~' in val:
        text = re.compile('[0-9]~').search(val).start()
        i_start = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).start() + 1
        i_end = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).end()
        ran = val[int(i_start):int(i_end)]
        for i in range(1, int(ran) + 1):
            val_lis2.append('{}{}'.format(val[:text], i))
            
    else:
        val_lis2.append(val)

df18 = df18.loc[:,val_lis2]

# 랜덤 배분 함수
def ratelist(row_name, val_rmd):
    li = []
    
    if row_name == 'Q9_2A1':
        dic = {1.0:0.318198, 2.0:0.214775, 3.0:0.134685, 10.0:0.101622,
                4.0:0.092523, 6.0:0.064324, 5.0:0.034054,7.0:0.017477,
                8.0:0.013423, 9.0:0.008919}
    elif row_name == 'Q9_2A2':
        dic = {2.0:0.328732,1.0:0.192887,3.0:0.145144,4.0:0.110057,
                10.0:0.072189,5.0:0.066053,7.0:0.031636,6.0:0.030678,
                8.0:0.011888,9.0:0.010737}
    else:
        dic = {3.0:0.176084,2.0:0.157057,1.0:0.156711,4.0:0.129958,
                10.0:0.127076,5.0:0.111624,6.0:0.054659,7.0:0.047970,
                8.0:0.024331,9.0:0.014530}
    
    rate = list(dic.values())
    total = len(df18[df18[row_name] == 21])
    count = 0

    for i in range(1,11):
        num = int(round(dic[i], len(str(total))) * total)
        count += num
        for j in range(0, num):
            li.append(i)
            
    if count != total:
        for k in range(0, total-count):
            li.append(int(list(dic.keys())[0]))
            
    random.shuffle(li)
    
    for l, m in zip(range(0, len(li)), df18[df18[row_name] == val_rmd].index):
        df18[row_name][m] = li[l]

# 18년 변수값 변경
val_lis=['Q2A1~Q2A5', 'Q5_2A1~Q5_2A3', 'Q9_2A1~Q9_2A3']
len_index = 0
df18['q2a1'] = np.nan
df18['q2a2'] = np.nan
df18['q2a3'] = np.nan
df18['q2a4'] = np.nan
df18['q2a5'] = np.nan
df18['Q2A1'] = df18['Q2A1'].fillna(1)

for val in val_lis:
    text = re.compile('[0-9]~').search(val).start()
    i_start = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).start()+1
    i_end = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).end()
    ran = val[int(i_start):int(i_end)]
    
    for i in range(1,int(ran)+1):
        name = '{}{}'.format(val[:text],i)
        
        if 'Q2A' in name:
            for j in range(0,len(df18)):
                if df18[name][j] == 1:
                    df18['q2a1'][j] = 1
                elif df18[name][j] == 2:
                    df18['q2a2'][j] = 2
                elif df18[name][j] == 3:
                    df18['q2a3'][j] = 3
                elif df18[name][j] == 4:
                    df18['q2a4'][j] = 4
                elif df18[name][j] == 5:
                    df18['q2a5'][j] = 5
        
        elif 'Q5_2A' in name:
            del_index = df18[df18[name]==4].index # del : 4:친구, 친지 방문
            len_index += len(del_index) # D_Q5_1 : 새 범주
            df18 = df18.drop(del_index)
            df18 = df18.reset_index(drop=True)
            df18[name] = df18[name].replace({5:4, 6:4, 8:5, 9:6, 10:8, 11:9, 12:11,
                                             13:12, 14:10, 15:3, 16:13, 17:13})

        elif 'Q9_2A' in name:
            df18[name] = df18[name].replace({8:21}) # 8임시값
            df18[name] = df18[name].replace({5:8, 7:8, 9:5, 10:5, 11:7, 12:14, 13:11, 14:12,
                                             15:13, 16:9, 17:9, 18:10, 19:10, 20:14})
            ratelist(name, 21) # 8 : 비율대로 분할

# 18년 Q2A1 컬럼명 변경
df18 = df18.drop(['Q2A1', 'Q2A2', 'Q2A3', 'Q2A4', 'Q2A5'], axis = 1)
df18.rename(columns = {'q2a1':'Q2A1', 'q2a2':'Q2A2', 'q2a3':'Q2A3', 'q2a4':'Q2A4', 'q2a5':'Q2A5'}, inplace = True)

# 18년 방한목적 변수값 기준에 맞게 변경
df18['D_Q5_1'] = df18['D_Q5_1'].replace({2:3, 3:4, 4:5, 5:6})
i_dq5 = random.sample(list(df18[df18['D_Q5_1'] == 1].index), len_index)
df18.loc[i_dq5,'D_Q5_1'] = 2.0

# 소비내역(통신비, 치료비) 기타로 통합
df18['Q14_2_89'] = df18['Q14_2_8'] + df18['Q14_2_9']
df18['Q14_2_89_1'] = np.nan
i = 0

for val in df18['Q14_2_89']:
    if val == 0:
        df18['Q14_2_89_1'][i] = 1
    elif val <= 30:
        df18['Q14_2_89_1'][i] = 2
    elif val <= 100:
        df18['Q14_2_89_1'][i] = 3
    elif val > 10000:
        df18['Q14_2_89_1'][i] = 98
    else:
        df18['Q14_2_89_1'][i] = 4
    i += 1

df18 = df18.drop(['Q14_2_8','Q14_2_9','Q14_2_89'], axis = 1)
df18.rename(columns = {'Q14_2_89_1' : 'Q14_2_9'}, inplace=True)

# 18년 여행지 변수명 한글로 변환
for i in range(357) : # 18년여행지변수명의 row 갯수가 357개
    df18 = df18.replace({'Q11A1' : df18_val.loc[i, '변수명']}, df18_val.loc[i, '변수내용'])
    df18 = df18.replace({'Q11A2' : df18_val.loc[i, '변수명']}, df18_val.loc[i, '변수내용'])
    df18 = df18.replace({'Q11A3' : df18_val.loc[i, '변수명']}, df18_val.loc[i, '변수내용'])
    
# 18년도 여행지 결측값 확인 후 제거
df18 = df18.drop(df18[df18['Q11A1'] == 998].index[0])
df18 = df18.drop(df18[df18['Q11A1'] == 998].index)
df18 = df18.drop(df18[df18['Q11A1'] == 309].index)
df18.reset_index(drop = True, inplace = True)

# 18년 컬럼명 19년도와 맞추기
dic_col={}
for i in range(len(df18_col)):
    dic_col.update({df18_col['18변수명'][i] : df17_col['19변수명'][i]})

df18.rename(columns = dic_col, inplace = True)

# 월이 들어간 컬럼을 사용해 년-월 컬럼 생성
for i in range(len(df18)) :
    df18.loc[i, "D_YEAR"] = "2018-{}".format(str(int(df18["D_MON"][i])).zfill(2))
    
# 외래방문객의 방문횟수 이상치 제거
df18 = df18.drop(df18[df18['Q1'] > 300].index).reset_index(drop = True)

# 19년 전처리

In [5]:
# 19년도 데이터 프레임 불러오기
val_lis = list(df_col['val1'].dropna(axis = 0))
val_lis2 = []

for val in val_lis:
    if '~' in val:
        text = re.compile('[0-9]~').search(val).start()
        i_start = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).start() + 1
        i_end = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).end()
        ran = val[int(i_start):int(i_end)]
        for i in range(1,int(ran) + 1):
            val_lis2.append('{}{}'.format(val[:text],i))
            
    else:
        val_lis2.append(val)

df19 = df19.loc[:,val_lis2]

# 동반자 현황 맞추기
val = 'Q2A1~Q2A5'
df19['q2a1'] = np.nan
df19['q2a2'] = np.nan
df19['q2a3'] = np.nan
df19['q2a4'] = np.nan
df19['q2a5'] = np.nan

text = re.compile('[0-9]~').search(val).start()
i_start = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).start() + 1
i_end = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).end()
ran = val[int(i_start):int(i_end)]

for i in range(1, int(ran) + 1):
    name = '{}{}'.format(val[:text], i)

    if 'Q2A' in name:
        for j in range(0, len(df19)):
            if df19[name][j] == 1:
                df19['q2a1'][j] = 1
            elif df19[name][j] == 2:
                df19['q2a2'][j] = 2
            elif df19[name][j] == 3:
                df19['q2a3'][j] = 3
            elif df19[name][j] == 4:
                df19['q2a4'][j] = 4
            elif df19[name][j] == 5:
                df19['q2a5'][j] = 5
                
df19['Q14_2_8910'] = df19['Q14_2_8'] + df19['Q14_2_9'] + df19['Q14_2_10']
df19['Q14_2_8910_1'] = np.nan

i = 0
for val in df19['Q14_2_8910']:
    if val == 0:
        df19['Q14_2_8910_1'][i] = 1
    elif val <= 30:
        df19['Q14_2_8910_1'][i] = 2
    elif val <= 100:
        df19['Q14_2_8910_1'][i] = 3
    elif val > 10000:
        df19['Q14_2_8910_1'][i] = 98
    else:
        df19['Q14_2_8910_1'][i] = 4
        
i += 1

# 필요없는 컬럼 제거, 정렬한 컬럼명 재설정
df19 = df19.drop(['Q2A1', 'Q2A2', 'Q2A3', 'Q2A4', 'Q2A5',
                  'Q14_2_8', 'Q14_2_9', 'Q14_2_10', 'Q14_2_8910'], axis = 1)
df19.rename(columns = {'q2a1':'Q2A1', 'q2a2':'Q2A2', 'q2a3':'Q2A3', 'q2a4':'Q2A4',
                       'q2a5':'Q2A5', 'Q14_2_8910_1':'Q14_2_10'}, inplace = True)

# 월이 들어간 컬럼을 사용해 년-월 컬럼 생성
for i in range(len(df19)) :
    df19.loc[i, "D_YEAR"] = "2019-{}".format(str(int(df19["D_MON"][i])).zfill(2))
    
for i in range(365) : # 19년여행지변수명의 row 갯수가 365개
    df19 = df19.replace({'Q10_2A1' : df19_val.loc[i, '변수명']}, df19_val.loc[i, '변수내용'])
    df19 = df19.replace({'Q10_2A2' : df19_val.loc[i, '변수명']}, df19_val.loc[i, '변수내용'])
    df19 = df19.replace({'Q10_2A3' : df19_val.loc[i, '변수명']}, df19_val.loc[i, '변수내용'])
    
# 외래방문객의 방문횟수 이상치 제거
df19 = df19.drop(df19[df19['Q1'] > 300].index).reset_index(drop = True)

# 데이터 통합

In [6]:
df17.to_csv('./data_new/2017년_변수정리.csv', encoding = 'utf-8', index = False)
df18.to_csv('./data_new/2018년_변수정리.csv', encoding = 'utf-8', index = False)
df19.to_csv('./data_new/2019년_변수정리.csv', encoding = 'utf-8', index = False)

In [7]:
df_tot = pd.concat([df17, df18, df19], ignore_index = True)

In [8]:
df_tot.to_csv('./data_new/2017-2019_통합본.csv', encoding = 'utf-8', index = False)